F12 找到 <head> 中的 <script>

In [ ]:

function curry( orig_func ) {
    var ap = Array.prototype, args = arguments;

    function fn() {
    ap.push.apply( fn.args, arguments );
    return fn.args.length < orig_func.length ? fn : orig_func.apply( this, fn.args );
    }

    return function() {
    fn.args = ap.slice.call( args, 1 );
    return fn.apply( this, arguments );
    };
}

function callback(x,y,i,a) {
  return !y.call(x, a[a["length"]-1-i].toString().slice(19,21)) ? x : {};
}

var ref = {T : "BG8",J : "jep",j : "M2L",K : "L23",H : "r1A"};

function validatekey()
{
    e = false;
    var _strKey = "";
    try {
        _strKey = document.getElementById("cpass").value;
        var a = _strKey.split("-");
        if(a.length !== 5)
            e = true;

        var o=a.map(genFunc).reduceRight(callback, new (genFunc(a[4]))(Function));

        if(!equal(o,ref))
            e = true;

    }catch(x){
        e = true;
    }

    if(!e) {
        if(document.location.href.indexOf("?key=") == -1) {
            document.location = document.location.href + "?key=" + _strKey;
        }
    } else {
        $("#error").html("<div class='alert alert-danger'><img src='error.png' style='width:100%'/></div>");
    }
}

function equal(o,o1)
{
    var keys1 = Object.keys(o1);
    var keys = Object.keys(o);
    if(keys1.length != keys.length)
        return false;

    for(var i=0;i<keys.length;i++)
         if(keys[i] != keys1[i] || o[keys[i]] != o1[keys1[i]])
            return false;

    return true;

}

function hook(f1,f2,f3) {
    return function(x) { return f2(f1(x),f3(x));};
}

var h = curry(hook);
var fn = h(function(x) {return x >= 48;},new Function("a","b","return a && b;"));
function genFunc(_part) {
    if(!_part || !(_part.length) || _part.length !== 4)
        return function() {};

    return new Function(_part.substring(1,3), "this." + _part[3] + "=" + _part.slice(1,3) + "+" + (fn(function(y){return y<=57})(_part.charCodeAt(0)) ?  _part[0] : "'"+ _part[0] + "'"));
}


### 分析內容  
從 `validatekey()` 知道 key 為 XXXX-XXXX-XXXX-XXXX-XXXX （每段4字，共5段）   
- 分析 `genFunc(_part)`: 會產生動態函數  
    - ex: `_part` = "ABCD"  
    - new Function("BC", "this.D=BC+ (數字判斷 ? A : "'" + A + "'")") --> ("BC", "this.D=BC+'A'")  
        等價 function F(BC) {this.D = BC + 'A';}  
- 分析 `reduceRight(callback, new genFunc(a[4])(Function))`   
    - `a.map(genFunc)`: 把每一段轉成 Function  
    - `callback`: 轉字串為「從右到左」  
    - `reduceRight(callback, obj)`: 從右到左合併每個函數  
- 如果結果 = {T : "BG8",J : "jep",j : "M2L",K : "L23",H : "r1A"} 就是對的  

// 有錯
### --> 反推: T: "BG8"  
    先反推 `reduceRight()` --> 8BGT  
    "BG" + 8 存到 T，所以 this.T = BG + 8 --> 符合 BG8  
### --> 反推: J: "jep"  
    先反推 `reduceRight()` --> pjepJ  
    "je" + 'p' 存到 J --> 符合 jep  
### --> 反推: j : "M2L"   
    先反推 `reduceRight()` --> LM2j   
    "M2" + 'L' 存到 j --> M22 不對，該是 "L"  
### --> 反推: K : "L23"  
    先反推 `reduceRight()` --> 3L2K    
    "L2" + 3 存到 K --> 符合 L23   
### --> 反推: H : "r1A"    
    先反推 `reduceRight()` --> Ar1H   
    "r1" + 'A' 存到 H --> 符合 r1A   
   

### --> key= 8BGT-pjeJ-LM2j-3L2K-Ar1H